Importing relavent packages for the project

In [243]:
import numpy as py
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

**Creating initial dataframes**

**Movie Character Dataframe**

In [244]:
column_names = ['chId', 'chName', 'mId', 'mName', 'gender', 'posCredits']
characters_df = pd.read_csv('movie_characters_metadata.tsv', sep='\t', header=None, names=column_names,error_bad_lines=False)
characters_df.head()


Skipping line 6565: expected 6 fields, saw 13



,chId,chName,mId,mName,gender,posCredits
0,u0,BIANCA,m0,10 things i hate about you,f,4
1,u1,BRUCE,m0,10 things i hate about you,?,?
2,u2,CAMERON,m0,10 things i hate about you,m,3
3,u3,CHASTITY,m0,10 things i hate about you,?,?
4,u4,JOEY,m0,10 things i hate about you,m,6


In [245]:
print(characters_df.shape)
print(characters_df.info())

(9034, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9034 entries, 0 to 9033
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   chId        9034 non-null   object
 1   chName      9015 non-null   object
 2   mId         9017 non-null   object
 3   mName       9017 non-null   object
 4   gender      9017 non-null   object
 5   posCredits  9017 non-null   object
dtypes: object(6)
memory usage: 423.6+ KB
None


In [246]:
nan_percentage = (characters_df.isna().mean() * 100).round(2)
print(nan_percentage)

chId          0.00
chName        0.21
mId           0.19
mName         0.19
gender        0.19
posCredits    0.19
dtype: float64


In [247]:
# Remove rows with NaN values
characters_df = characters_df.dropna()

In [248]:
# Remove duplicate rows
duplicate_counts = characters_df.duplicated().sum()
print("Total duplicate", duplicate_counts)
if duplicate_counts > 0:
  characters_df = characters_df.drop_duplicates()

Total duplicate 0


Cleansing Column Gender

In [249]:
characters_df.gender.value_counts()

?    6006
m    1899
f     921
M     145
F      44
Name: gender, dtype: int64

In [250]:
characters_df = characters_df[characters_df.gender != '?']
characters_df.gender = characters_df.gender.apply(lambda g: 'M' if g in ['m', 'M'] else 'F')
characters_df.gender.value_counts()

M    2044
F     965
Name: gender, dtype: int64

Cleansing Column posCredits

In [251]:
non_numeric_count = pd.to_numeric(characters_df['posCredits'], errors='coerce').isna().sum()
# Display the total count of non-numeric values
print("Total count of non-numeric values in 'posCredits' removed {}".format(non_numeric_count))

# Remove rows with non-numeric values in the specified column
characters_df = characters_df[pd.to_numeric(characters_df['posCredits'], errors='coerce').notna()]

Total count of non-numeric values in 'posCredits' removed 330


**Movie title Dataframe**

In [252]:
column_names = ['mId', 'mName', 'mYear', 'mRating', 'mVotes', 'mGenre']
titles_df = pd.read_csv('movie_titles_metadata.tsv', sep='\t', names=column_names, header=None, error_bad_lines=False)
titles_df.head()

,mId,mName,mYear,mRating,mVotes,mGenre
0,m0,10 things i hate about you,1999,6.9,62847.0,['comedy' 'romance']
1,m1,1492: conquest of paradise,1992,6.2,10421.0,['adventure' 'biography' 'drama' 'history']
2,m2,15 minutes,2001,6.1,25854.0,['action' 'crime' 'drama' 'thriller']
3,m3,2001: a space odyssey,1968,8.4,163227.0,['adventure' 'mystery' 'sci-fi']
4,m4,48 hrs.,1982,6.9,22289.0,['action' 'comedy' 'crime' 'drama' 'thriller']


In [253]:
print(titles_df.shape)
print(titles_df.info())


(617, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   mId      617 non-null    object 
 1   mName    616 non-null    object 
 2   mYear    616 non-null    object 
 3   mRating  616 non-null    float64
 4   mVotes   616 non-null    float64
 5   mGenre   616 non-null    object 
dtypes: float64(2), object(4)
memory usage: 29.0+ KB
None


Removing rows with NAN

In [254]:
nan_percentage = (titles_df.isna().mean() * 100).round(2)
print(nan_percentage)

mId        0.00
mName      0.16
mYear      0.16
mRating    0.16
mVotes     0.16
mGenre     0.16
dtype: float64


In [255]:
# Remove rows with NaN values
titles_df = titles_df.dropna()

In [256]:
# Remove duplicate rows
duplicate_counts = titles_df.duplicated().sum()
print("Total Number of duplicate rows", duplicate_counts)
if duplicate_counts > 0:
  titles_df = titles_df.drop_duplicates()

Total Number of duplicate rows 0


No outier data found in numeric columns

In [257]:
titles_df.describe()

,mRating,mVotes
count,616.000000,616.000000
mean,6.865584,49901.698052
std,1.215463,61898.367352
min,2.500000,9.000000
25%,6.200000,9992.500000
50%,7.000000,27121.500000
75%,7.800000,66890.000000
max,9.300000,419312.000000


Extracting the year part from the column mYear

In [258]:
column_name = 'mYear' 

# Filter non-numeric values in the specified column
non_numeric_values = titles_df.loc[~pd.to_numeric(titles_df[column_name], errors='coerce').notna(), column_name]

# Get unique non-numeric values
unique_non_numeric_values = non_numeric_values.unique()

# Display the unique non-numeric values
print("Unique non-numeric values in column '{}':".format(column_name))
print(unique_non_numeric_values)

Unique non-numeric values in column 'mYear':
['1989/I' '1990/I' '1995/I' '1998/I' '2004/I' '2007/I' '1992/I' '2005/I'
 '2002/I' '1968/I' '1996/I' '2000/I' '2009/I' '2003/I']


In [259]:
column_name = 'mYear'

# Extract numeric portion using regular expression
titles_df[column_name] = titles_df[column_name].str.extract('(\d+)', expand=False)
titles_df[column_name] = titles_df[column_name].astype(int)


In [260]:
titles_df.head()

,mId,mName,mYear,mRating,mVotes,mGenre
0,m0,10 things i hate about you,1999,6.9,62847.0,['comedy' 'romance']
1,m1,1492: conquest of paradise,1992,6.2,10421.0,['adventure' 'biography' 'drama' 'history']
2,m2,15 minutes,2001,6.1,25854.0,['action' 'crime' 'drama' 'thriller']
3,m3,2001: a space odyssey,1968,8.4,163227.0,['adventure' 'mystery' 'sci-fi']
4,m4,48 hrs.,1982,6.9,22289.0,['action' 'comedy' 'crime' 'drama' 'thriller']


Movie Lines

In [261]:
column_names = ['lId', 'chId', 'mId', 'chName', 'chLine']
lines_df = pd.read_csv('movie_lines.tsv', sep='\t', header=None,names=column_names, warn_bad_lines=False,error_bad_lines=False)
lines_df.head()

,lId,chId,mId,chName,chLine
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.


In [262]:
print(lines_df.shape)
print(lines_df.info())

(293202, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293202 entries, 0 to 293201
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   lId     293202 non-null  object
 1   chId    288917 non-null  object
 2   mId     288917 non-null  object
 3   chName  288874 non-null  object
 4   chLine  288663 non-null  object
dtypes: object(5)
memory usage: 11.2+ MB
None


In [263]:
nan_percentage = (lines_df.isna().mean() * 100).round(2)
print(nan_percentage)

lId       0.00
chId      1.46
mId       1.46
chName    1.48
chLine    1.55
dtype: float64


In [264]:
lines_df = lines_df.dropna()

In [265]:
# Remove duplicate rows
duplicate_counts = lines_df.duplicated().sum()
print("Total duplicates: ",duplicate_counts)
if duplicate_counts > 0:
  lines_df = lines_df.drop_duplicates()

Total duplicates:  0


In [266]:
print(lines_df.shape)

(288620, 5)


In [267]:
column_names = ['chId1', 'chId2','mId', 'lineList']
conversations_df = pd.read_csv('movie_conversations.tsv', sep='\t',names=column_names, header=None,warn_bad_lines=False,error_bad_lines=False)
conversations_df.head()

,chId1,chId2,mId,lineList
0,u0,u2,m0,['L194' 'L195' 'L196' 'L197']
1,u0,u2,m0,['L198' 'L199']
2,u0,u2,m0,['L200' 'L201' 'L202' 'L203']
3,u0,u2,m0,['L204' 'L205' 'L206']
4,u0,u2,m0,['L207' 'L208']


In [268]:
nan_percentage = (conversations_df.isna().mean() * 100).round(2)
print(nan_percentage)

chId1       0.0
chId2       0.0
mId         0.0
lineList    0.0
dtype: float64


In [269]:
# Remove duplicate rows
duplicate_counts = lines_df.duplicated().sum()
print("Total Number Of Duplicates: ", duplicate_counts)
if duplicate_counts > 0:
  lines_df = lines_df.drop_duplicates()

Total Number Of Duplicates:  0


In [270]:
conversations_df.shape

(83097, 4)

In [271]:
conversations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83097 entries, 0 to 83096
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   chId1     83097 non-null  object
 1   chId2     83097 non-null  object
 2   mId       83097 non-null  object
 3   lineList  83097 non-null  object
dtypes: object(4)
memory usage: 2.5+ MB


In [272]:
column_names = ['mId','mName','url']
rawScript_df = pd.read_csv('raw_script_urls.tsv', sep='\t', names=column_names, header=None,warn_bad_lines=False,error_bad_lines=False)
rawScript_df.head()

,mId,mName,url
0,m0,10 things i hate about you,http://www.dailyscript.com/scripts/10Things.html
1,m1,1492: conquest of paradise,http://www.hundland.org/scripts/1492-ConquestO...
2,m2,15 minutes,http://www.dailyscript.com/scripts/15minutes.html
3,m3,2001: a space odyssey,http://www.scifiscripts.com/scripts/2001.txt
4,m4,48 hrs.,http://www.awesomefilm.com/script/48hours.txt


In [273]:
rawScript_df.shape

(617, 3)

In [274]:
nan_percentage = (rawScript_df.isna().mean() * 100).round(2)
print(nan_percentage)

mId      0.00
mName    0.16
url      0.16
dtype: float64


In [275]:
# Remove rows with NaN values
rawScript_df = rawScript_df.dropna()

In [276]:
# Remove duplicate rows
duplicate_counts = rawScript_df.duplicated().sum()
print("Total Number Of Duplicates: ", duplicate_counts)
if duplicate_counts > 0:
  rawScript_df = rawScript_df.drop_duplicates()

Total Number Of Duplicates:  0


In [277]:
rawScript_df.shape

(616, 3)